In [526]:
import requests
from bs4 import BeautifulSoup

import pandas as pd, numpy as np
import re

## getting air force base locations

In [310]:
airforce_wiki_url = "https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations"
df_usaf = pd.read_html(airforce_wiki_url)[1]

In [350]:
df_usaf.head(3)

,Name,Location,State,Coordinates,Commanding organization,Wing or unit emblem,Host wing or primary unit,Primary missions and units,lat,lon
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W﻿ / ﻿34.66639°N 99.26806°W,Air Education and Training Command,NaN,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34.66639,-99.26806
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W﻿ / ﻿38.84278°N 77.01611°W,Air Force District of Washington,NaN,11th Wing,"US Navy operated joint base, accommodating Geo...",38.84278,-77.01611
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W﻿ / ﻿38.81083°N 76.86694°W,Air Force District of Washington,NaN,11th Wing,USAF operated joint base. The 11th Wing provid...,38.81083,-76.86694


In [312]:
def extract_coords(s):
    # extract coordinates from the Coordinates column in the table
    s = s.split('/')[1]
    # remove BOM from beginning of string
    s = s.replace('\ufeff', '')
    s = s.strip()
    # remove all chars except decimals and NSEW letter
    coords = ''.join(item for item in s if item in ' .1234567890NSEW').split()
    # remove NSEW designation from each coord, convert to float, and assign negative if needed
    coords = [float(coord[:-1]) if coord[-1] in 'NE' else -float(coord[:-1]) for coord in coords]
    return coords

In [319]:
coords = pd.DataFrame(df_usaf['Coordinates'].apply(extract_coords).to_list(), columns = ['lat', 'lon'])

In [320]:
df_usaf = df_usaf.join(coords)

In [349]:
df_usaf.head(3)

,Name,Location,State,Coordinates,Commanding organization,Wing or unit emblem,Host wing or primary unit,Primary missions and units,lat,lon
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W﻿ / ﻿34.66639°N 99.26806°W,Air Education and Training Command,NaN,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34.66639,-99.26806
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W﻿ / ﻿38.84278°N 77.01611°W,Air Force District of Washington,NaN,11th Wing,"US Navy operated joint base, accommodating Geo...",38.84278,-77.01611
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W﻿ / ﻿38.81083°N 76.86694°W,Air Force District of Washington,NaN,11th Wing,USAF operated joint base. The 11th Wing provid...,38.81083,-76.86694


make sure were not missing any coordinates

In [479]:
df_usaf.lat.isna().value_counts()

False    70
Name: lat, dtype: int64

## getting postal office locations

In [174]:
from io import StringIO

In [252]:
def gather_USPS_data():

    # create session and specify header due to cookies issue with the about.usps.com site
    s = requests.Session()
    headers = {
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'en-US,en;q=0.8',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
    }
    r = s.get("https://about.usps.com/who/legal/foia/owned-facilities.htm", headers = headers)
    l = soup.find_all('ul', class_='list-unstyled list-multi-column')[0].find_all('li')

    # for each state csv in the html table, create a dataframe and append to list to be concat after
    dfs = []
    for item in l:
        # create path using html 
        path = "https://about.usps.com" + item.find('a').get('href')
        r_csv = s.get(path, headers = headers)
        # remove some misc text from the beginning of the string 
        r_csv = r_csv.text.replace("""Owned Area - Building Inventory 2,,,,,,,,,,,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,,,,,\r\n""", '')
        st = StringIO(r_csv)
        # drop the last two rows, which are misc 
        df = pd.read_csv(st)[:-2]
        dfs.append(df)

    final_df = pd.concat(dfs, ignore_index = True)
    return final_df

In [496]:
df_usps = gather_USPS_data()

In [497]:
df_usps.head(3)

,District,Fin-Sub,Chrgbl Fin No,PO Name,Unit Name,Property Address,County,City,ST,ZIP Code,...,Ownership,FDB ID (All),AMS Locale Key (All),FDB Facility Type (All),FDB Facility Subtype (All),Building Ownership Description,Land Desc,Space Certified Indicator,Bldg Occu Date,Int Sq Ft
0,Alabama,010120-G02,10120.0,ALBERTVILLE,MAIN OFFICE,210 S HAMBRICK ST,MARSHALL,ALBERTVILLE,AL,35950-1624,...,Owned,1352597.0,Y10022,Post Office,Main Post Office,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",No,12/1/1983,"8,913"
1,Alabama,010150-G03,10150.0,ALEXANDER CITY,MAIN OFFICE,233 LEE ST,TALLAPOOSA,ALEXANDER CITY,AL,35010-2654,...,Owned,1352654.0,Y10026,Post Office,Administrative Post Office (APO),"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,9/1/1984,"7,748"
2,Alabama,010240-G01,10240.0,ALTON,MAIN OFFICE MODULAR,5548 JOHNSON ST,JEFFERSON,ALTON,AL,35015-2001,...,Owned,1352868.0,Y10035,Post Office,Remotely Managed Post Office (RMPO),"USPS Building, Not Prev. Leased",Land Data on separate record,No,9/1/1995,672


In [498]:
# make sure to drop duplicates that exist across all columns...
df_usps.duplicated().value_counts()

False    9418
True     9308
dtype: int64

In [499]:
df_usps = df_usps.drop_duplicates()

### write out chunks of df_usps no larger than 10k records each

needs example format:

1,4600 Silver Hill Rd,Suitland,MD,20746  
2,436 15th St SE, Washington, DC,20003  
  


key columns are Street, City, State, Zip

For geocoding purposes, make sure there are no duplicates on the key columns in order to reduce queries


In [440]:
key_cols = ['Property Address', 'City', 'ST', 'ZIP Code']

In [501]:
df_usps[key_cols].duplicated().value_counts()

False    8343
True     1075
dtype: int64

thats a ton of dupes ... locations might have more than one USPS-role and therefore show up multiple times

In [502]:
uniques = df_usps[key_cols].drop_duplicates()

In [503]:
uniques

,Property Address,City,ST,ZIP Code
0,210 S HAMBRICK ST,ALBERTVILLE,AL,35950-1624
1,233 LEE ST,ALEXANDER CITY,AL,35010-2654
2,5548 JOHNSON ST,ALTON,AL,35015-2001
3,520 E THREE NOTCH ST,ANDALUSIA,AL,36420-3128
4,7312 HIGHWAY 207,ANDERSON,AL,35610-4840
...,...,...,...,...
17968,US HIGHWAY 14N,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17969,US HIGHWAY 89/191/287,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17970,US HIGHWAY 14/16 and US HIGHWAY 89-191,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17971,1000 MAMMOTH,YELLOWSTONE NATIONAL PARK,WY,82190-9650


write out in chunks just in case the # of uniques is > 10000

In [341]:
chunk_size = 10000

for i in range(round(len(uniques)/chunk_size)):
    chunk = uniques[i*chunk_size:chunk_size*(i+1)]
    chunk.to_csv("data/uscensus/usps_chunk_{}.csv".format(i), header = False)

In [358]:
# pass to geocoding service
# https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
!curl --form addressFile=@data/uscensus/usps_chunk_0.csv --form benchmark=Public_AR_Current https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output data/uscensus/geocoderesult.csv

% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1316k  100  946k  100  369k   1201    468  0:13:27  0:13:27 --:--:--  146k


In [363]:
df_geocoderesult = pd.read_csv("data/uscensus/geocoderesult.csv", index_col = 0, names = ['input', 'match', 'match_type', 'output', 'coords', 'tigerlineID', 'side'])

In [364]:
df_geocoderesult.head()

,input,match,match_type,output,coords,tigerlineID,side
17288,"22433 RANDOLPH DR, DULLES, VA, 20104-9998",Match,Non_Exact,"22433 RANDOLPH DR, DULLES, VA, 20103","-77.45251,38.995213",62362877.0,R
17289,"44715 PRENTICE DR, DULLES, VA, 20101-9998",Match,Non_Exact,"44715 PRENTICE DR, DULLES, VA, 20166","-77.4544,39.001156",62362878.0,L
17284,"10001 COUNTY DR, DISPUTANTA, VA, 23842-9998",Match,Exact,"10001 COUNTY DR, DISPUTANTA, VA, 23842","-77.22715,37.124813",613894685.0,L
17285,"22365 DREWRY RD, DREWRYVILLE, VA, 23844-9998",Match,Exact,"22365 DREWRY RD, DREWRYVILLE, VA, 23844","-77.30634,36.715645",82709745.0,R
4970,"345 E SUNSET BLVD, GERLACH, NV, 89412-9800",Tie,NaN,NaN,NaN,NaN,NaN


In [365]:
df_geocoderesult.coords.isna().value_counts()

False    5355
True     2988
Name: coords, dtype: int64

need to look into JSON request per line to try and geocode remainder


for now, lets try to look at what we have

In [431]:
geocode_lookup = df_geocoderesult[~df_geocoderesult.coords.isna()][['input', 'coords']]

In [432]:
def fix_coords(coord):
    # rearrage coords 
    lon, lat = coord.split(',')
    return float(lat), float(lon)

def split_input(s):
    street, city, state, zip9 = [item.strip() for item in s.split(',')]
    return street, city, state, zip9

In [433]:
split_input = geocode_lookup.input.apply(split_input).apply(pd.Series)
split_input.columns =  key_cols
split_coords = geocode_lookup.coords.apply(fix_coords).apply(pd.Series)
split_coords.columns = ['lat', 'lon']

In [435]:
split_coords

,lat,lon
17288,38.995213,-77.452510
17289,39.001156,-77.454400
17284,37.124813,-77.227150
17285,36.715645,-77.306340
17286,37.093610,-80.686910
...,...,...
4954,40.867040,-97.591820
3629,39.291080,-76.623825
4955,41.244614,-96.396126
3627,39.328594,-76.631240


In [436]:
geocode_result = split_input.join(split_coords)

In [437]:
geocode_result

,Property Address,City,ST,ZIP Code,lat,lon
17288,22433 RANDOLPH DR,DULLES,VA,20104-9998,38.995213,-77.452510
17289,44715 PRENTICE DR,DULLES,VA,20101-9998,39.001156,-77.454400
17284,10001 COUNTY DR,DISPUTANTA,VA,23842-9998,37.124813,-77.227150
17285,22365 DREWRY RD,DREWRYVILLE,VA,23844-9998,36.715645,-77.306340
17286,1 TOWN CENTER DR,DUBLIN,VA,24084-9998,37.093610,-80.686910
...,...,...,...,...,...,...
4954,626 N GRANT AVE,YORK,NE,68467-9998,40.867040,-97.591820
3629,130 N GREENE ST,BALTIMORE,MD,21201-9997,39.291080,-76.623825
4955,502 1ST ST,YUTAN,NE,68073-9700,41.244614,-96.396126
3627,919 W 34TH ST,BALTIMORE,MD,21211-9998,39.328594,-76.631240


now perform left merge back with the original df_usps using the key cols

In [627]:
df_usps_final = df_usps.merge(geocode_result, how = 'left', on = key_cols)

In [628]:
df_usps_final.head()

,District,Fin-Sub,Chrgbl Fin No,PO Name,Unit Name,Property Address,County,City,ST,ZIP Code,...,AMS Locale Key (All),FDB Facility Type (All),FDB Facility Subtype (All),Building Ownership Description,Land Desc,Space Certified Indicator,Bldg Occu Date,Int Sq Ft,lat,lon
0,Alabama,010120-G02,10120.0,ALBERTVILLE,MAIN OFFICE,210 S HAMBRICK ST,MARSHALL,ALBERTVILLE,AL,35950-1624,...,Y10022,Post Office,Main Post Office,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",No,12/1/1983,"8,913",NaN,NaN
1,Alabama,010150-G03,10150.0,ALEXANDER CITY,MAIN OFFICE,233 LEE ST,TALLAPOOSA,ALEXANDER CITY,AL,35010-2654,...,Y10026,Post Office,Administrative Post Office (APO),"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,9/1/1984,"7,748",NaN,NaN
2,Alabama,010240-G01,10240.0,ALTON,MAIN OFFICE MODULAR,5548 JOHNSON ST,JEFFERSON,ALTON,AL,35015-2001,...,Y10035,Post Office,Remotely Managed Post Office (RMPO),"USPS Building, Not Prev. Leased",Land Data on separate record,No,9/1/1995,672,NaN,NaN
3,Alabama,010270-G01,10270.0,ANDALUSIA,MAIN OFFICE,520 E THREE NOTCH ST,COVINGTON,ANDALUSIA,AL,36420-3128,...,Y10038,Post Office,Administrative Post Office (APO),"USPS Building, Prev. Leased","USPS Land, Prev. Leased",Yes,10/1/1965,"10,519",31.302319,-86.487960
4,Alabama,010280-G01,10280.0,ANDERSON,MAIN OFFICE MODULAR,7312 HIGHWAY 207,LAUDERDALE,ANDERSON,AL,35610-4840,...,Y10039,Post Office,Remotely Managed Post Office (RMPO),USPS Personal Property,Land Data on separate record,No,4/1/1995,992,34.924915,-87.268715


In [629]:
df_usps_final.lat.isna().value_counts()

False    6097
True     3321
Name: lat, dtype: int64

not bad, missing roughly 1/3 of total lat/lon coords tho

In [630]:
df_usps_final.head(1).T

,0
District,Alabama
Fin-Sub,010120-G02
Chrgbl Fin No,10120
PO Name,ALBERTVILLE
Unit Name,MAIN OFFICE
Property Address,210 S HAMBRICK ST
County,MARSHALL
City,ALBERTVILLE
ST,AL
ZIP Code,35950-1624


should probably look into what types of USPS facilities we're interested in, and filter out the others

In [635]:
df_usps_final['FDB Facility Type (All)'].value_counts()

Post Office                            8266
Administrative Office                   316
Mail Processing                         302
Vehicle Maintenance                     250
Network Facilities                       40
Other Customer Service                   21
Training Facility                         6
Storage Facility                          2
International Operations Facilities       2
Other Operations Facilities               1
Unknown Type                              1
Name: FDB Facility Type (All), dtype: int64

In [636]:
df_usps_interest = df_usps_final[df_usps_final['FDB Facility Type (All)'].isin(('Mail Processing', 'Network Facilities'))]

In [638]:
# write out for corey
df_usps_interest.to_excel("data/outputs/usps_locations_interest.xlsx")

In [645]:
(df_usps_interest.lat.isna() & df_usps_interest.lat.isna()).value_counts()

False    234
True     108
Name: lat, dtype: int64

In [646]:
df_usps_interest_coded = df_usps_interest[~df_usps_interest.lat.isna()]
df_usps_interest_notcoded = df_usps_interest[df_usps_interest.lat.isna()]

### use GeoPy to attempt to geocode the remaining missing lat/lon

In [692]:
from geopy.geocoders import Nominatim

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [694]:
geolocator = Nominatim(user_agent = "DEAN", timeout=10)

In [660]:
df_usps_interest_notcoded.head()

,District,Fin-Sub,Chrgbl Fin No,PO Name,Unit Name,Property Address,County,City,ST,ZIP Code,...,AMS Locale Key (All),FDB Facility Type (All),FDB Facility Subtype (All),Building Ownership Description,Land Desc,Space Certified Indicator,Bldg Occu Date,Int Sq Ft,lat,lon
213,Alaska,024173-G02,24177.0,JUNEAU,MENDENHALL STATION,9491 VINTAGE BLVD,JUNEAU,JUNEAU,AK,99801-7111,...,Z10149,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,8/1/1986,"29,212",NaN,NaN
222,Alaska,024563-G02,24563.0,KETCHIKAN,MAIN OFFICE,3609 TONGASS AVE,PRINCE OF WALES-OUTER KETCHIKAN,KETCHIKAN,AK,99901-9998,...,Z10160,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,9/1/1976,"16,950",NaN,NaN
242,Alaska,026357-G03,26357.0,NOME,MAIL PROCESSING ANNEX,516 PORT RD,NOME,NOME,AK,99762-9801,...,1203,Mail Processing,Mail Processing Annex (ANX),"USPS Building, Const. by USPS",Land Data on separate record,No,3/1/1987,"8,118",NaN,NaN
351,Arizona,036365-G15,36365.0,PHOENIX,P&DC,4949 E VAN BUREN ST RM 65,MARICOPA,PHOENIX,AZ,85026-9600,...,Z10746,Network Facilities,Network Distribution Center (NDC/ASF),"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,6/1/1985,"473,016",NaN,NaN
353,Arizona,036365-G15,36365.0,PHOENIX,P&DC,4949 E VAN BUREN ST RM 65,MARICOPA,PHOENIX,AZ,85026-9600,...,Z10745,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,6/1/1985,"473,016",NaN,NaN


In [652]:
df_usps_interest_notcoded[['Property Address', 'City', 'ST', 'ZIP Code']].apply(lambda row: ', '.join(row), axis = 1)

213             9491 VINTAGE BLVD, JUNEAU, AK, 99801-7111
222           3609 TONGASS AVE, KETCHIKAN, AK, 99901-9998
242                     516 PORT RD, NOME, AK, 99762-9801
351     4949 E VAN BUREN ST RM 65, PHOENIX, AZ, 85026-...
353     4949 E VAN BUREN ST RM 65, PHOENIX, AZ, 85026-...
                              ...                        
9057           2928 S SPOTTED RD, SPOKANE, WA, 99224-9997
9069               4001 S PINE ST, TACOMA, WA, 98413-9994
9184             1025 W 20TH AVE, OSHKOSH, WI, 54902-9998
9255              200 CAVA DR, CLARKSBURG, WV, 26301-9993
9356              411 N FOREST DR, CASPER, WY, 82609-9997
Length: 108, dtype: object

In [657]:
df_usps_interest_notcoded[['Property Address','City','ST','ZIP Code']].apply(lambda row: ', '.join(row), axis = 1)

213             9491 VINTAGE BLVD, JUNEAU, AK, 99801-7111
222           3609 TONGASS AVE, KETCHIKAN, AK, 99901-9998
242                     516 PORT RD, NOME, AK, 99762-9801
351     4949 E VAN BUREN ST RM 65, PHOENIX, AZ, 85026-...
353     4949 E VAN BUREN ST RM 65, PHOENIX, AZ, 85026-...
                              ...                        
9057           2928 S SPOTTED RD, SPOKANE, WA, 99224-9997
9069               4001 S PINE ST, TACOMA, WA, 98413-9994
9184             1025 W 20TH AVE, OSHKOSH, WI, 54902-9998
9255              200 CAVA DR, CLARKSBURG, WV, 26301-9993
9356              411 N FOREST DR, CASPER, WY, 82609-9997
Length: 108, dtype: object

In [689]:
address_cols = ['Property Address','City','ST','ZIP Code']
def geocode_single(row):
    address = ', '.join(row[item] for item in address_cols)
    location = geolocator.geocode(address)
    if location:
        print('found')
        return location.latitude, location.longitude
    else:
        print('trying city')
        address = ', '.join(row[item] for item in address_cols[1:])
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            print('trying state')
            address = ', '.join(row[item] for item in address_cols[2:])
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                print('FAILED')
                return None, None

In [690]:
test = df_usps_interest_notcoded.copy()

In [706]:
df_usps_interest_notcoded[['lat', 'lon']] = test.coord.apply(pd.Series)

In [709]:
df_usps_interest_final = pd.concat((df_usps_interest_notcoded, df_usps_interest_coded))

In [710]:
df_usps_interest_final

,District,Fin-Sub,Chrgbl Fin No,PO Name,Unit Name,Property Address,County,City,ST,ZIP Code,...,AMS Locale Key (All),FDB Facility Type (All),FDB Facility Subtype (All),Building Ownership Description,Land Desc,Space Certified Indicator,Bldg Occu Date,Int Sq Ft,lat,lon
213,Alaska,024173-G02,24177.0,JUNEAU,MENDENHALL STATION,9491 VINTAGE BLVD,JUNEAU,JUNEAU,AK,99801-7111,...,Z10149,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,8/1/1986,"29,212",58.372074,-134.597132
222,Alaska,024563-G02,24563.0,KETCHIKAN,MAIN OFFICE,3609 TONGASS AVE,PRINCE OF WALES-OUTER KETCHIKAN,KETCHIKAN,AK,99901-9998,...,Z10160,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,9/1/1976,"16,950",55.354713,-131.690471
242,Alaska,026357-G03,26357.0,NOME,MAIL PROCESSING ANNEX,516 PORT RD,NOME,NOME,AK,99762-9801,...,1203,Mail Processing,Mail Processing Annex (ANX),"USPS Building, Const. by USPS",Land Data on separate record,No,3/1/1987,"8,118",64.501111,-165.406389
351,Arizona,036365-G15,36365.0,PHOENIX,P&DC,4949 E VAN BUREN ST RM 65,MARICOPA,PHOENIX,AZ,85026-9600,...,Z10746,Network Facilities,Network Distribution Center (NDC/ASF),"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,6/1/1985,"473,016",33.448437,-112.074142
353,Arizona,036365-G15,36365.0,PHOENIX,P&DC,4949 E VAN BUREN ST RM 65,MARICOPA,PHOENIX,AZ,85026-9600,...,Z10745,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,6/1/1985,"473,016",33.448437,-112.074142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9149,Lakeland,564981-G02,564981.0,MADISON,P&DC,3902 MILWAUKEE ST,DANE,MADISON,WI,53714-3000,...,Y17610,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,2/1/1976,"257,027",43.098305,-89.319190
9226,Lakeland,568696-G03,568696.0,WAUSAU,P&DF,400 CRESKE AVE,MARATHON,ROTHSCHILD,WI,54474-7955,...,Y18274,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,6/1/1997,"55,709",44.863544,-89.626976
9248,Appalachian,551459-G08,551459.0,CHARLESTON,P&DC,1000 CENTRE WAY,KANAWHA,CHARLESTON,WV,25309-9426,...,X28291,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,10/1/1993,"220,900",38.313515,-81.710810
9358,Colorado/Wyoming,571673-G05,571673.0,CHEYENNE,P&DC,4800 CONVERSE AVE,LARAMIE,CHEYENNE,WY,82009-9997,...,W1B336,Mail Processing,Processing and Distribution Center/Facility (P...,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,5/1/1995,"116,338",41.163950,-104.786100


In [696]:
test['coord'] = test[['Property Address','City','ST','ZIP Code']].apply(geocode_single, axis = 1)

found
found
trying city
trying city
trying city
trying city
found
trying city
trying state
found
found
found
found
trying city
found
found
found
found
found
trying city
trying city
trying city
trying city
found
trying city
found
found
trying city
trying state
found
found
found
trying city
found
found
found
found
found
found
trying city
trying city
found
trying city
found
trying city
found
trying city
found
found
found
found
found
found
found
found
found
trying city
found
found
found
found
trying city
found
trying city
found
trying city
trying city
found
found
found
found
found
found
trying city
trying city
trying city
found
found
found
found
found
found
found
trying city
trying city
found
trying city
trying city
trying city
found
trying city
found
found
found
found
found
found
trying city
found
found
found
trying city
found
found
found
found
found
found
found
found


## perform cross join between USAF and USPS locations

In [457]:
def crossjoin(left_df, right_df, suffixes = ('_x', '_y')):
    # cartesian product
    return pd.merge(left_df.assign(key=0), right_df.assign(key=0), on='key', suffixes = suffixes).drop('key', axis = 1)

In [711]:
# create a unique key for each column in usps dataframe
df_usps_interest_final['USPS_KEY'] = pd.util.hash_pandas_object(df_usps_interest_final, index = False)

In [712]:
# as a key, use the Base Name for USAF and the USPS_KEY for usps
df_usaf_latlon = df_usaf[['Name', 'lat', 'lon']]
df_usps_latlon = df_usps_interest_final[['USPS_KEY', 'lat', 'lon']]

In [713]:
df_crossjoin = crossjoin(df_usaf_latlon, df_usps_latlon, suffixes = ('_USAF', '_USPS'))

In [714]:
df_crossjoin

,Name,lat_USAF,lon_USAF,USPS_KEY,lat_USPS,lon_USPS
0,Altus Air Force Base,34.66639,-99.26806,1502536841536567391,58.372074,-134.597132
1,Altus Air Force Base,34.66639,-99.26806,15259707190130931823,55.354713,-131.690471
2,Altus Air Force Base,34.66639,-99.26806,13997465678146455466,64.501111,-165.406389
3,Altus Air Force Base,34.66639,-99.26806,11301300975455368198,33.448437,-112.074142
4,Altus Air Force Base,34.66639,-99.26806,5886277595280375461,33.448437,-112.074142
...,...,...,...,...,...,...
23935,Wright-Patterson Air Force Base,39.82306,-84.04944,12431445325323235737,43.098305,-89.319190
23936,Wright-Patterson Air Force Base,39.82306,-84.04944,7805317990284307633,44.863544,-89.626976
23937,Wright-Patterson Air Force Base,39.82306,-84.04944,12191199484822242347,38.313515,-81.710810
23938,Wright-Patterson Air Force Base,39.82306,-84.04944,17246579629023441205,41.163950,-104.786100


use haversine to calculate great circle distance between each coordinate pair

In [528]:
def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km

In [715]:
df_crossjoin['dist'] = haversine(df_crossjoin.lat_USAF, df_crossjoin.lon_USAF, df_crossjoin.lat_USPS, df_crossjoin.lon_USPS)

In [716]:
df_crossjoin.head()

,Name,lat_USAF,lon_USAF,USPS_KEY,lat_USPS,lon_USPS,dist
0,Altus Air Force Base,34.66639,-99.26806,1502536841536567391,58.372074,-134.597132,3698.292209
1,Altus Air Force Base,34.66639,-99.26806,15259707190130931823,55.354713,-131.690471,3378.863014
2,Altus Air Force Base,34.66639,-99.26806,13997465678146455466,64.501111,-165.406389,5443.770683
3,Altus Air Force Base,34.66639,-99.26806,11301300975455368198,33.448437,-112.074142,1186.655356
4,Altus Air Force Base,34.66639,-99.26806,5886277595280375461,33.448437,-112.074142,1186.655356


In [717]:
df_result = df_crossjoin[['Name', 'USPS_KEY', 'dist']]

In [718]:
df_result.head()

,Name,USPS_KEY,dist
0,Altus Air Force Base,1502536841536567391,3698.292209
1,Altus Air Force Base,15259707190130931823,3378.863014
2,Altus Air Force Base,13997465678146455466,5443.770683
3,Altus Air Force Base,11301300975455368198,1186.655356
4,Altus Air Force Base,5886277595280375461,1186.655356


In [719]:
# lets dedup on distance for now ... will worry about aggregating info for USPS locations later
df_result = df_result.drop_duplicates('dist')

In [722]:
df_result.head()

,Name,USPS_KEY,dist
0,Altus Air Force Base,1502536841536567391,3698.292209
1,Altus Air Force Base,15259707190130931823,3378.863014
2,Altus Air Force Base,13997465678146455466,5443.770683
3,Altus Air Force Base,11301300975455368198,1186.655356
5,Altus Air Force Base,7290112359649404019,567.550827


In [761]:
df_final

dist
Name                            USPS_KEY                          
Altus Air Force Base            14264900933901804976    178.432672
                                8248659727851441256     239.540645
                                2063121342049035601     266.049941
                                17082669835612363717    272.326393
                                10122557200728567212    281.916445
...                                                            ...
Wright-Patterson Air Force Base 12833727596271742205   4959.107811
                                13997465678146455466   5684.013356
                                8602778705132184377    6907.946022
                                11270454408986530537   7163.446804
                                11562124430031091637  12232.186749

[21910 rows x 1 columns]

In [754]:
df_final = df_result.sort_values(['Name', 'dist']).set_index(['Name', 'USPS_KEY'])

In [763]:
df_final_stats.head(10)

,count,mean,std,min,25%,50%,75%,max
Name,,,,,,,,
Altus Air Force Base,313.0,1614.874245,1005.312641,178.432672,1099.915518,1493.381414,2033.725632,11401.720985
Barksdale Air Force Base,313.0,1563.515038,1098.259352,9.996008,965.468271,1388.241918,1973.748087,11973.064695
Beale Air Force Base,313.0,2698.219914,1258.830285,51.990056,2004.923242,2906.330400,3632.719568,9384.054802
Buckley Air Force Base,313.0,1731.316430,974.390096,16.130327,1215.214140,1594.330474,2285.931017,10725.181645
Cannon Air Force Base,313.0,1790.144820,996.147933,159.929989,1283.471337,1693.340343,2155.967944,11078.597244
Cape Canaveral Air Force Station,313.0,1991.044069,1367.921182,74.174326,1236.390504,1609.404781,2446.211603,13263.545841
Cape Cod Air Force Station,313.0,2050.602891,1458.532496,54.400424,1078.908526,1744.740282,2835.704657,12885.633840
Cavalier Air Force Station,313.0,1777.624848,873.226169,112.152824,1317.605278,1796.536803,2108.986298,10766.709533
Columbus Air Force Base,313.0,1460.768767,1205.832306,13.965319,778.730016,1159.752761,1740.094012,12321.244593


In [762]:
df_final_stats = df_final.groupby('Name')['dist'].describe()

## write stuff to excel workbooks

In [606]:
df_usaf.to_excel("data/outputs/usaf_locations.xlsx")

In [607]:
df_usps_final.to_excel("data/outputs/usps_locations.xlsx")

In [610]:
# # old, not used anymore
# df_final.to_excel("data/outputs/distance_nsmallest_{}.xlsx".format(n_smallest))

In [765]:
df_final_stats.to_excel("data/outputs/distances_stats.xlsx")

In [764]:
df_final.to_excel("data/outputs/distances_all.xlsx")